# Отток клиентов

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. 

Описание данных:
1. RowNumber — индекс строки в данных
2. CustomerId — уникальный идентификатор клиента
3. Surname — фамилия
4. CreditScore — кредитный рейтинг
5. Geography — страна проживания
6. Gender — пол
7. Age — возраст
8. Tenure — сколько лет человек является клиентом банка
9. Balance — баланс на счёте
10. NumOfProducts — количество продуктов банка, используемых клиентом
11. HasCrCard — наличие кредитной карты
12. IsActiveMember — активность клиента
13. EstimatedSalary — предполагаемая зарплата
14. **Exited** — факт ухода клиента

## Подготовка данных

In [1]:
import warnings
warnings.filterwarnings("ignore") 
import pandas as pd 
from sklearn.model_selection import train_test_split 

from sklearn.metrics import roc_auc_score 
from sklearn.metrics import f1_score 
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier  
from sklearn.linear_model import LogisticRegression  
from sklearn.model_selection import GridSearchCV 
from sklearn.preprocessing import StandardScaler  
from sklearn.utils import shuffle 

In [2]:
data_frame=pd.read_csv('/datasets/Churn.csv') # загрузка данных
data_frame.info()
display(data_frame.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [3]:
null_data_columns=data_frame.isna().sum() 
columns_data_old=list(data_frame.columns) 
display(null_data_columns)
null_data_columns=list(null_data_columns) 

print('Доля пропусков в столбце:') 
for i in range(len(columns_data_old)):
    rezul=1-(len(data_frame)-null_data_columns[i])/len(data_frame)
    print(columns_data_old[i], '=', f'{rezul:.2%}' )

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

Доля пропусков в столбце:
RowNumber = 0.00%
CustomerId = 0.00%
Surname = 0.00%
CreditScore = 0.00%
Geography = 0.00%
Gender = 0.00%
Age = 0.00%
Tenure = 9.09%
Balance = 0.00%
NumOfProducts = 0.00%
HasCrCard = 0.00%
IsActiveMember = 0.00%
EstimatedSalary = 0.00%
Exited = 0.00%


In [4]:
print('Доля пропусков в столбце ,%:') 
rezul=data_frame.isna().mean() * 100
print(rezul)

Доля пропусков в столбце ,%:
RowNumber          0.00
CustomerId         0.00
Surname            0.00
CreditScore        0.00
Geography          0.00
Gender             0.00
Age                0.00
Tenure             9.09
Balance            0.00
NumOfProducts      0.00
HasCrCard          0.00
IsActiveMember     0.00
EstimatedSalary    0.00
Exited             0.00
dtype: float64


**Вывод** 
1. Примерно 9% клиентов не имеют времени использования услуг банка. Скорее всего это новые клиенты которые пользуются банком менее 1 года, эти данные можно удалить.
2. Так же можно удалить признаки не имеющее отношение к решению уйти из банка, это RowNumber, CustomerId, Surname

In [5]:
data_frame = data_frame.dropna(subset=['Tenure']) 
data_frame = data_frame.drop(['RowNumber', 'CustomerId','Surname'], axis=1) 
null_data_columns=data_frame.isna().sum() 
display(null_data_columns)
display(data_frame.head(10))

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [6]:
data_ohe=pd.get_dummies(data_frame,drop_first=True)  # признаки переводятся в численные
display(data_ohe.head(10))

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0
5,645,44,8.0,113755.78,2,1,0,149756.71,1,0,1,1
6,822,50,7.0,0.00,2,1,1,10062.80,0,0,0,1
7,376,29,4.0,115046.74,4,1,0,119346.88,1,1,0,0
8,501,44,4.0,142051.07,2,0,1,74940.50,0,0,0,1
9,684,27,2.0,134603.88,1,1,1,71725.73,0,0,0,1


Разделение на выборки

In [7]:
features =data_ohe.drop(['Exited',], axis=1)  
target =data_ohe['Exited']  
# разделяем на выборку для обучения и тестирования, 25% данных для валидации (теста)
features_train, features_data, target_train, target_data = train_test_split(features, target, test_size=0.4, random_state=2504)
features_test, features_valid, target_test, target_valid = train_test_split( features_data, target_data, test_size=0.5, random_state=2504)


numeric = ['CreditScore','Age','Tenure','Balance','EstimatedSalary']
scaler = StandardScaler() 
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric]=scaler.transform(features_valid[numeric])
features_test[numeric]=scaler.transform(features_test[numeric])
pd.options.mode.chained_assignment = None

## Исследование задачи

### Логическая классификация

In [8]:
%%time
model_LR =  LogisticRegression(random_state=2504,solver='liblinear')
model_LR.fit(features_train, target_train) 
predicted_valid = model_LR.predict(features_valid)

print('Точность  =',accuracy_score(target_valid, predicted_valid))
print('Метрика F1 =',f1_score(target_valid, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_valid,predicted_valid))

Точность  = 0.8147333699835074
Метрика F1 = 0.32464929859719444
Метрика roc_auc = 0.593210689544804
CPU times: user 36 ms, sys: 27.2 ms, total: 63.2 ms
Wall time: 21.6 ms


**Вывод**
Линейная регрессия не дает необходимый уровень точности.

### Дерево решений

In [9]:
%%time
model_DTC =  DecisionTreeClassifier()
parametrs={
    'max_depth':range(1,10,2),
    'min_samples_split':range(1,8),
    'min_samples_leaf':range(2,10,2)
}
grid_param=GridSearchCV(model_DTC,parametrs,cv=3)
grid_param.fit(features_train, target_train)

CPU times: user 4.27 s, sys: 31.3 ms, total: 4.3 s
Wall time: 4.37 s


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 10, 2),
                         'min_samples_leaf': range(2, 10, 2),
                         'min_samples_split': range(1, 8)})

Лучшие параметры обучения

In [10]:
grid_param.best_params_

{'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 6}

In [11]:
model_DTC = DecisionTreeClassifier(random_state=2504, max_depth=grid_param.best_params_['max_depth']).fit(features_train,target_train) # обучение модели с заданной глубиной дерева
predicted_valid= model_DTC.predict(features_valid) # получение предсказания модели
print('Точность  =',accuracy_score(target_valid,predicted_valid))
print('Метрика F1 =',f1_score(target_valid, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_valid,predicted_valid))

Точность  = 0.8460692688290269
Метрика F1 = 0.5483870967741935
Метрика roc_auc = 0.7001521244072298


**Вывод**
Дерево решений не дает необходимый уровень точности.

### Случайный лес

In [12]:
%%time
model_RFC = RandomForestClassifier()
parametrs={
    'n_estimators':range(10,61,10),
    'max_depth':range(1,13,2),
    'min_samples_split':range(1,8),
    'min_samples_leaf':range(2,10,2)
}
grid_param=GridSearchCV(model_RFC,parametrs,cv=3)
grid_param.fit(features_train, target_train)


CPU times: user 5min 6s, sys: 1.79 s, total: 5min 8s
Wall time: 5min 8s


GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'max_depth': range(1, 13, 2),
                         'min_samples_leaf': range(2, 10, 2),
                         'min_samples_split': range(1, 8),
                         'n_estimators': range(10, 61, 10)})

Лучшие параметры обучения

In [13]:
grid_param.best_params_

{'max_depth': 11,
 'min_samples_leaf': 2,
 'min_samples_split': 4,
 'n_estimators': 50}

In [14]:
model_RFC = RandomForestClassifier(random_state=2504, max_depth=grid_param.best_params_['max_depth'],n_estimators=grid_param.best_params_['n_estimators']).fit(features_train, target_train) # обучение модели с заданной глубиной дерева
predicted_valid = model_RFC.predict(features_valid) # получение предсказания модели
print('Точность  =', accuracy_score(target_valid, predicted_valid))
print('Метрика F1 =',f1_score(target_valid, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_valid,predicted_valid))

Точность  = 0.8510170423309511
Метрика F1 = 0.528695652173913
Метрика roc_auc = 0.6856414425882119


**Вывод**
Дерево решений не дает необходимый уровень точности.

### Выводы

Из-за дисбаланса классов невозможно добиться требуемой точности в имеющихся моделях машинного обучения, хуже всего показал себя метод логической классификации

## Борьба с дисбалансом

### Увеличение выборки (upsampling)

In [8]:
def upsemp (features_1,target_1,repeat):
    features_zeros = features_1[target_1 == 0]
    features_ones = features_1[target_1 == 1]
    target_zeros = target_1[target_1 == 0]
    target_ones = target_1[target_1 == 1]
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    features_train_upsampled, target_train_upsampled = shuffle(features_upsampled, target_upsampled, random_state=2504)
    return features_upsampled, target_upsampled
features_train_upsemp, target_train_upsemp=upsemp(features_train,target_train,5)

### Уменьшение выборки (downsampling)

In [9]:
def downsamp (features_1, target_1, fraction):
    features_zeros = features_1[target_1 == 0]
    features_ones = features_1[target_1 == 1]
    target_zeros = target_1[target_1 == 0]
    target_ones = target_1[target_1 == 1]
    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=2504)] + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=2504)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=2504)
    return features_downsampled, target_downsampled
features_train_downsamp, target_train_downsamp=downsamp(features_train,target_train,0.1)

### Логическая классификация

Взвешивание классов

In [17]:
%%time
model_LR_b =  LogisticRegression(random_state=2504,solver='liblinear',class_weight='balanced')
model_LR_b.fit(features_train, target_train) 
predicted_valid = model_LR_b.predict(features_valid)

print('Точность  =',accuracy_score(target_valid, predicted_valid))
print('Метрика F1 =',f1_score(target_valid, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_valid,predicted_valid))

Точность  = 0.7157778999450247
Метрика F1 = 0.4995159728944821
Метрика roc_auc = 0.7041722923878934
CPU times: user 48.6 ms, sys: 15.9 ms, total: 64.5 ms
Wall time: 22.3 ms


Увеличение выборки (upsampling)

In [18]:
%%time
model_LR_up =  LogisticRegression(random_state=2504,solver='liblinear')
model_LR_up.fit(features_train_upsemp, target_train_upsemp) # обучение модели 
predicted_valid = model_LR_up.predict(features_valid)# получение предсказания модели

print('Точность  =',accuracy_score(target_valid, predicted_valid))
print('Метрика F1 =',f1_score(target_valid, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_valid,predicted_valid))

Точность  = 0.6586036283672347
Метрика F1 = 0.4872006606110653
Метрика roc_auc = 0.7043534804666374
CPU times: user 65.8 ms, sys: 54.3 ms, total: 120 ms
Wall time: 111 ms


Уменьшение выборки (downsampling)

In [19]:
%%time
model_LR_down =  LogisticRegression(random_state=2504,solver='liblinear')
model_LR_down.fit(features_train_downsamp, target_train_downsamp) # обучение модели 
predicted_valid = model_LR_down.predict(features_valid)# получение предсказания модели

print('Точность  =',accuracy_score(target_valid, predicted_valid))
print('Метрика F1 =',f1_score(target_valid, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_valid,predicted_valid))

Точность  = 0.4667399670148433
Метрика F1 = 0.4205495818399044
Метрика roc_auc = 0.6391735984136386
CPU times: user 20.2 ms, sys: 23.9 ms, total: 44.1 ms
Wall time: 17.7 ms


**Вывод**

Логическая классификация после баланса классов показывает метрики удовлетворяющие требованию

### Дерево решений

Взвешивание классов

In [20]:
%%time
model_DTC =  DecisionTreeClassifier()
parametrs={
    'max_depth':range(1,10,2),
    'min_samples_split':range(1,8),\
    'min_samples_leaf':range(2,10,2)
}
grid_param=GridSearchCV(model_DTC,parametrs,cv=3)
grid_param.fit(features_train, target_train)

CPU times: user 4.44 s, sys: 55.8 ms, total: 4.49 s
Wall time: 4.51 s


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 10, 2),
                         'min_samples_leaf': range(2, 10, 2),
                         'min_samples_split': range(1, 8)})

In [21]:
grid_param.best_params_

{'max_depth': 5, 'min_samples_leaf': 4, 'min_samples_split': 2}

In [22]:
model_DTC_b = DecisionTreeClassifier(random_state=2504, max_depth=grid_param.best_params_['max_depth'],class_weight='balanced')
model_DTC_b.fit(features_train,target_train) 
predicted_valid= model_DTC_b.predict(features_valid) 
print('Точность  =',accuracy_score(target_valid, predicted_valid))
print('Метрика F1 =',f1_score(target_valid, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_valid, predicted_valid))

Точность  = 0.736668499175371
Метрика F1 = 0.5577100646352724
Метрика roc_auc = 0.7604472862256592


Увеличение выборки (upsampling)

In [23]:
%%time
model_DTC_up =  DecisionTreeClassifier()
parametrs={
    'max_depth':range(1,27,2),
    'min_samples_split':range(1,8),
    'min_samples_leaf':range(2,10,2)
}
grid_param=GridSearchCV(model_DTC_up,parametrs,cv=3)
grid_param.fit(features_train_upsemp, target_train_upsemp)

CPU times: user 21.7 s, sys: 83.9 ms, total: 21.8 s
Wall time: 21.8 s


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 27, 2),
                         'min_samples_leaf': range(2, 10, 2),
                         'min_samples_split': range(1, 8)})

In [24]:
grid_param.best_params_

{'max_depth': 25, 'min_samples_leaf': 2, 'min_samples_split': 4}

In [26]:
model_DTC_up = DecisionTreeClassifier(random_state=2504,max_depth=grid_param.best_params_['max_depth']).fit(features_train_upsemp,target_train_upsemp) # обучение модели с заданной глубиной дерева
predicted_valid= model_DTC_up.predict(features_valid) # получение предсказания модели
print('Точность  =',accuracy_score(target_valid, predicted_valid))
print('Метрика F1 =',f1_score(target_valid, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_valid, predicted_valid))

Точность  = 0.8020890599230346
Метрика F1 = 0.5068493150684931
Метрика roc_auc = 0.6871056630012105


Уменьшение выборки (downsampling)

In [27]:
%%time
model_DTC_down =  DecisionTreeClassifier()
parametrs={
    'max_depth':range(1,33,2),
    'min_samples_split':range(1,8),
    'min_samples_leaf':range(2,10,2)
}
grid_param=GridSearchCV(model_DTC_down,parametrs,cv=3)
grid_param.fit(features_train_downsamp, target_train_downsamp)

CPU times: user 8.5 s, sys: 3.99 ms, total: 8.5 s
Wall time: 8.51 s


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 33, 2),
                         'min_samples_leaf': range(2, 10, 2),
                         'min_samples_split': range(1, 8)})

Лучшие параметры обучения

In [28]:
grid_param.best_params_

{'max_depth': 3, 'min_samples_leaf': 2, 'min_samples_split': 2}

In [30]:
model_DTC_down = DecisionTreeClassifier(random_state=2504, max_depth=grid_param.best_params_['max_depth']).fit(features_train_downsamp,target_train_downsamp) # обучение модели с заданной глубиной дерева
predicted_valid= model_DTC_down.predict(features_valid) # получение предсказания модели
print('Точность  =',accuracy_score(target_valid,predicted_valid))
print('Метрика F1 =',f1_score(target_valid, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_valid,predicted_valid))

Точность  = 0.5316107751511819
Метрика F1 = 0.44675324675324674
Метрика roc_auc = 0.6722528392263913


**Вывод**

Классификация при помощи дерева решений позволяет добиться приемлемого уровня метрик точности

### Случайный лес

Взвешивание классов

In [31]:
%%time
model_RFC_b = RandomForestClassifier()
parametrs={
    'n_estimators':range(10,61,10),
    'max_depth':range(1,13,2),
    'min_samples_split':range(1,8),
    'min_samples_leaf':range(2,10,2)
}
grid_param=GridSearchCV(model_RFC_b,parametrs,cv=3)
grid_param.fit(features_train, target_train)


CPU times: user 5min 15s, sys: 1.66 s, total: 5min 17s
Wall time: 5min 17s


GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'max_depth': range(1, 13, 2),
                         'min_samples_leaf': range(2, 10, 2),
                         'min_samples_split': range(1, 8),
                         'n_estimators': range(10, 61, 10)})

In [32]:
grid_param.best_params_

{'max_depth': 11,
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 60}

In [10]:
model_RFC_b = RandomForestClassifier(random_state=2504, max_depth=grid_param.best_params_['max_depth'],n_estimators=grid_param.best_params_['n_estimators'],class_weight='balanced').fit(features_train, target_train) # обучение модели с заданной глубиной дерева
predicted_valid = model_RFC_b.predict(features_valid) 
print('Точность  =', accuracy_score(target_valid, predicted_valid))
print('Метрика F1 =',f1_score(target_valid, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_valid,predicted_valid))

Точность  = 0.8416712479384277
Метрика F1 = 0.5801749271137026
Метрика roc_auc = 0.7257842592626658


Увеличение выборки (upsampling)

In [34]:
%%time
model_RFC_up = RandomForestClassifier()
parametrs={
    'n_estimators':range(10,61,10),
    'max_depth':range(1,13,2),
    'min_samples_split':range(1,8),
    'min_samples_leaf':range(2,10,2)
}
grid_param=GridSearchCV(model_RFC_up,parametrs,cv=3)
grid_param.fit(features_train_upsemp, target_train_upsemp)


CPU times: user 7min 3s, sys: 2.73 s, total: 7min 6s
Wall time: 7min 6s


GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'max_depth': range(1, 13, 2),
                         'min_samples_leaf': range(2, 10, 2),
                         'min_samples_split': range(1, 8),
                         'n_estimators': range(10, 61, 10)})

In [35]:
grid_param.best_params_

{'max_depth': 11,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 60}

In [11]:
model_RFC_up = RandomForestClassifier(random_state=2504, max_depth=grid_param.best_params_['max_depth'],n_estimators=grid_param.best_params_['n_estimators']).fit(features_train_upsemp, target_train_upsemp) # обучение модели с заданной глубиной дерева
predicted_valid =model_RFC_up.predict(features_valid) # получение предсказания модели
print('Точность  =', accuracy_score(target_valid, predicted_valid))
print('Метрика F1 =',f1_score(target_valid, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_valid,predicted_valid))

Точность  = 0.822979659153381
Метрика F1 = 0.6129807692307694
Метрика roc_auc = 0.7688481588715939


Уменьшение выборки (downsampling)

In [38]:
%%time
model_RFC_down = RandomForestClassifier()
parametrs={
    'n_estimators':range(10,61,10),
    'max_depth':range(1,13,2),
    'min_samples_split':range(1,8),
    'min_samples_leaf':range(2,10,2)
}
grid_param=GridSearchCV(model_RFC_down,parametrs,cv=3)
grid_param.fit(features_train_downsamp, target_train_downsamp)

CPU times: user 3min 4s, sys: 1.79 s, total: 3min 6s
Wall time: 3min 6s


GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'max_depth': range(1, 13, 2),
                         'min_samples_leaf': range(2, 10, 2),
                         'min_samples_split': range(1, 8),
                         'n_estimators': range(10, 61, 10)})

In [39]:
grid_param.best_params_

{'max_depth': 9,
 'min_samples_leaf': 2,
 'min_samples_split': 7,
 'n_estimators': 40}

In [40]:
model_RFC_down = RandomForestClassifier(random_state=2504, max_depth=grid_param.best_params_['max_depth'],n_estimators=grid_param.best_params_['n_estimators']).fit(features_train_downsamp, target_train_downsamp) # обучение модели с заданной глубиной дерева
predicted_valid = model_RFC_down.predict(features_valid) # получение предсказания модели
print('Точность  =', accuracy_score(target_valid, predicted_valid))
print('Метрика F1 =',f1_score(target_valid, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_valid,predicted_valid))

Точность  = 0.577240241891149
Метрика F1 = 0.4743677375256324
Метрика roc_auc = 0.7039708701074621


**Вывод**

Классификация при помощи случайного леса позволяет добиться приемлемого значения метрик точности, но вслучае необходимости переобучения модели процесс занимает значительно больше времени в сравнении с другими методами.

### Выводы

Применение балансировки классов позволило добиться улучшения показателей метрик точности всех моделей. Наименее эффективным оказался метод баланса классов, использование этого подхода позволило только моделе случайного леса добиться требуемых метрик. Применение увеличеной и уменьшеной выборок позволило добиться требуемых метрик всем тестированным моделям. 

## Тестирование модели

### Логическая классификация

Взвешивание классов

In [41]:
predicted_valid = model_LR_b.predict(features_test)# получение предсказания модели
print('Точность  =',accuracy_score(target_test, predicted_valid))
print('Метрика F1 =',f1_score(target_test, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_test,predicted_valid))

Точность  = 0.7117711771177118
Метрика F1 = 0.4801587301587302
Метрика roc_auc = 0.6996687622004643


Увеличение выборки (upsampling)

In [42]:
predicted_valid = model_LR_up.predict(features_test)# получение предсказания модели
print('Точность  =',accuracy_score(target_test, predicted_valid))
print('Метрика F1 =',f1_score(target_test, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_test,predicted_valid))

Точность  = 0.6512651265126512
Метрика F1 = 0.4636209813874789
Метрика roc_auc = 0.6960489709340753


Уменьшение выборки (downsampling)

In [43]:
predicted_valid = model_LR_down.predict(features_test)# получение предсказания модели
print('Точность  =',accuracy_score(target_test, predicted_valid))
print('Метрика F1 =',f1_score(target_test, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_test,predicted_valid))

Точность  = 0.43014301430143015
Метрика F1 = 0.38987043580683156
Метрика roc_auc = 0.6191284065233096


### Дерево решений

Взвешивание классов

In [44]:
predicted_valid = model_DTC_b.predict(features_test)# получение предсказания модели
print('Точность  =',accuracy_score(target_test, predicted_valid))
print('Метрика F1 =',f1_score(target_test, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_test,predicted_valid))

Точность  = 0.7497249724972497
Метрика F1 = 0.5560975609756098
Метрика roc_auc = 0.7689539494919995


Увеличение выборки (upsampling)

In [45]:
predicted_valid = model_DTC_up.predict(features_test)# получение предсказания модели
print('Точность  =',accuracy_score(target_test, predicted_valid))
print('Метрика F1 =',f1_score(target_test, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_test,predicted_valid))

Точность  = 0.8008800880088008
Метрика F1 = 0.505464480874317
Метрика roc_auc = 0.6945099832459767


Уменьшение выборки (downsampling)

In [46]:
predicted_valid = model_DTC_down.predict(features_test)# получение предсказания модели
print('Точность  =',accuracy_score(target_test, predicted_valid))
print('Метрика F1 =',f1_score(target_test, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_test,predicted_valid))

Точность  = 0.5253025302530253
Метрика F1 = 0.4348395546823837
Метрика roc_auc = 0.6793564303170967


### Случайный лес

Взвешивание классов

In [12]:
predicted_valid = model_RFC_b.predict(features_test)# получение предсказания модели
print('Точность  =',accuracy_score(target_test, predicted_valid))
print('Метрика F1 =',f1_score(target_test, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_test,predicted_valid))

Точность  = 0.8597359735973598
Метрика F1 = 0.6188340807174888
Метрика roc_auc = 0.7544786270923316


Увеличение выборки (upsampling)

In [48]:
predicted_valid = model_RFC_up.predict(features_test)# получение предсказания модели
print('Точность  =',accuracy_score(target_test, predicted_valid))
print('Метрика F1 =',f1_score(target_test, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_test,predicted_valid))

Точность  = 0.8261826182618262
Метрика F1 = 0.6146341463414634
Метрика roc_auc = 0.7814291642970227


Уменьшение выборки (downsampling)

In [49]:
predicted_valid = model_RFC_down.predict(features_test)# получение предсказания модели
print('Точность  =',accuracy_score(target_test, predicted_valid))
print('Метрика F1 =',f1_score(target_test, predicted_valid))
print('Метрика roc_auc =', roc_auc_score(target_test,predicted_valid))

Точность  = 0.5588558855885588
Метрика F1 = 0.45737483085250336
Метрика roc_auc = 0.7065932461304354
